# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing dataset!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing data again. This time, however, we did things slightly different:
- We dropped `'ZN'` and `'NOX'` completely 
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable.
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and logtransformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and logtransformed all of them first, except for `'AGE'`) 

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'],axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0, 6, 24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD','TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [4]:
age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [10]:
boston_target = pd.DataFrame(boston.target, columns=['price'])

## Run an linear model in Statsmodels

In [11]:
boston_df = pd.concat([boston_features, boston_target], axis=1)
boston_df.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(270, 360]","TAX_(360, 712]",price
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,0,1,0,24.0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,0,0,0,21.6
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,0,0,0,34.7
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,0,0,0,33.4
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,0,0,0,36.2


In [21]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Store predictor variables and target variable in different names
preds = boston_df.drop('price', axis=1)

# Need the line of code directly below to obtain the intercept (constant) from OLS
preds_int = sm.add_constant(preds)
target = boston_df['price']

# run sm.OLS - this only requires input from target variable and predictors no 'formula' needed
sm_model = sm.OLS(target, preds_int).fit()
sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Sat, 19 Oct 2019   Prob (F-statistic):          5.08e-153
Time:                        13:46:48   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              8.6442      3.189      2.711      0.007       2.379      14.910
CRIM              -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS             -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS               2.5959      0.796      3.260      0.001       1.032       4.160
RM                 2.6466      0.408      6.488      0.000       1.845       3.448
AGE                0.0794      0.352      0.226      0.821      -0.612       0.770
DIS              -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO           -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B                  3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT             -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_(6, 24]        1.3380      0.672      1.990      0.047       0.017       2.659
TAX_(270, 360]    -1.2598      0.600     -2.100      0.036      -2.438      -0.081
TAX_(360, 712]    -2.1461      0.704     -3.047      0.002      -3.530      -0.762
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                         117.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Run the same model in Scikit-learn

In [22]:
# Your code here - Check that the coefficients and intercept are the same as those from Statsmodels
from sklearn.linear_model import LinearRegression

In [25]:
sk_model = LinearRegression().fit(preds, target)
display(sk_model.coef_)
display(sk_model.intercept_)

array([ -1.95380233,  -0.80457549,   2.59586776,   2.64657111,
         0.07939727, -10.09618465,  -1.48666599,   3.8412139 ,
        -5.62879369,   1.33796317,  -1.25977612,  -2.14606188])

8.644156137983725

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

#### House price tends to go up when the following variables increase: proximity to Charles River (CHAS), number of rooms (RM), age of home (AGE), B, higher RAD value.

#### House price tends to go down when the following variables increase: crime rate (CRIM), industrial area (INDUS), weighted distance to Boston employment centers (DIS), pupil-teacher ratio (PTRATIO), % lower status of population (LSTAT), and higher TAX.

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [37]:
# transform/standardize/normalize variables as we did previously to make our prediction

t_crim = np.log(0.15)
t_indus = np.log(6.07)
t_chas = 1
t_rm = 6.1
t_age = 33.2
t_dis = np.log(7.6)
t_ptratio = np.log(17)
t_b = 383
t_lstat = np.log(10.87)
t_rad_6_24 = 1
t_tax_270_360 = 1
t_tax_360_712 = 0
t_vals = np.array([t_crim, t_indus, t_chas, t_rm, t_age, t_dis,
                   t_ptratio, t_b, t_lstat, t_rad_6_24, t_tax_270_360, t_tax_360_712])

[ -1.89711998   1.80335861   1.           6.1         33.2
   2.02814825   2.83321334 383.           2.3860067    1.
   1.           0.        ]


In [38]:
price_pred = sk_model.intercept_ + np.dot(sk_model.coef_, t_vals)

#Print predicted price (in $10,000's)
print(price_pred)

1465.4199276137804


In [40]:
boston_df.describe()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(270, 360]","TAX_(360, 712]",price
count,506.000000,5.060000e+02,506.000000,506.000000,5.060000e+02,506.000000,5.060000e+02,506.000000,5.060000e+02,506.000000,506.000000,506.000000,506.000000
mean,0.448432,3.510587e-16,0.069170,6.284634,-1.474446e-16,0.449191,-2.878681e-15,0.898568,2.808469e-16,0.341897,0.343874,0.460474,22.532806
std,0.226336,1.000990e+00,0.253994,0.702617,1.000990e+00,0.227318,1.000990e+00,0.230205,1.000990e+00,0.474815,0.475470,0.498929,9.197104
min,0.000000,-3.783365e+00,0.000000,3.561000,-2.335437e+00,0.000000,-3.000989e+00,0.000000,-3.036568e+00,0.000000,0.000000,0.000000,5.000000
25%,0.268367,-6.614858e-01,0.000000,5.885500,-8.374480e-01,0.261281,-4.125447e-01,0.945730,-7.200364e-01,0.000000,0.000000,0.000000,17.025000
50%,0.387692,1.428756e-01,0.000000,6.208500,3.173816e-01,0.439687,3.139590e-01,0.986232,9.850402e-02,0.000000,0.000000,0.000000,21.200000
75%,0.666445,9.478254e-01,0.000000,6.623500,9.067981e-01,0.642307,7.840469e-01,0.998298,7.656165e-01,1.000000,1.000000,1.000000,25.000000
max,1.000000,1.497881e+00,1.000000,8.780000,1.117494e+00,1.000000,1.468580e+00,1.000000,2.108674e+00,1.000000,1.000000,1.000000,50.000000


## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing data.